# 데이터 로드하기

보스턴 주택 가격 데이터셋을 사용

train data : 404개

test data : 102개

feature : 13 개

\* feature마다 scale이 다름 \*

In [ ]:
from keras.datasets import boston_housing

(train_data,train_targets),(test_data,test_targets)=boston_housing.load_data()

In [ ]:
print("train data shape:",train_data.shape)
print("train targets shape",train_targets.shape)

In [ ]:
train_data[0]

In [ ]:
train_targets

# 데이터 전처리하기

## Scale 맞추기
feature별로 scale 다름 => __feature별로 정규화(normalization)__

- 정규화 공식 : (val-평균)/표준편차

In [ ]:
# 행 기준
mean=train_data.mean(axis=0)
std=train_data.std(axis=0)

print("feature별 평균 벡터 길이:",len(mean))
print("feature별 표준편차 벡터 길이:",len(std))

# 한번에 하는 것보다 이게 빠른가?
train_data-=mean
train_data/=std

## test data scale 맞추기

train data의 평균과 표준 편차를 이용해야한다

절대 test data의 정보를 이용하면 안된다!

In [ ]:
test_data-=mean
test_data/=std

# 모델 구성

## 함수 build_model() 정의 :

동일한 모델을 여러번 생성할 것이므로 함수로 정의한다

In [ ]:
from keras import models
from keras import layers

def build_model():
    
    model=models.Sequential()
    
    model.add(layers.Dense(64,activation='relu',input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.Dense(1))
    
    model.compile(optimizer='rmsprop',loss='mse',metrics=['mae'])
    
    return model

# 훈련,검증 하기

## k-fold cross-validation 

train data 많지 않으므로 k-fold cross-validation 이용

데이터를 k개로 분할하고

k개의 모델을 각각 만들어 k-1개의 분할에서 훈련하고 나머지 분할에서 평가하는 방법

모델의 검증 점수는 k개의 검증 점수의 평균이다

In [ ]:
import numpy as np

# 보통 4,5
k=4
num_val_samples=len(train_data)//k
num_epochs=100
all_scores=[]

for i in range(k):
    
    print("처리중인 폴드 #",i,end=' ')
    
    # validation data
    val_data=train_data[i*num_val_samples:(i+1)*num_val_samples]
    val_targets=train_targets[i*num_val_samples:(i+1)*num_val_samples]
    
    # train data
    partial_train_data=np.concatenate([train_data[:i*num_val_samples],train_data[(i+1)*num_val_samples:]],axis=0)
    partial_train_targets=np.concatenate([train_targets[:i*num_val_samples],train_targets[(i+1)*num_val_samples:]],axis=0)
    
    print("학습 시작....",end=' ')
    # 모델 학습
    model=build_model()
    # verbose=0 이면 훈련과정 출력되지 않는다
    model.fit(partial_train_data,partial_train_targets,epochs=num_epochs,batch_size=1,verbose=0)
    print("학습 끝!")
    
    # 모델 평가
    val_mse,val_mae=model.evaluate(val_data,val_targets,verbose=0)
    all_scores.append(val_mae)
    

In [ ]:
np.mean(all_scores)

In [ ]:
import numpy as np

# 보통 4,5
k=4
num_val_samples=len(train_data)//k
num_epochs=200
all_mae_histories=[]

for i in range(k):
    
    print("처리중인 폴드 #",i,end=' ')
    
    # validation data
    val_data=train_data[i*num_val_samples:(i+1)*num_val_samples]
    val_targets=train_targets[i*num_val_samples:(i+1)*num_val_samples]
    
    # train data
    partial_train_data=np.concatenate([train_data[:i*num_val_samples],train_data[(i+1)*num_val_samples:]],axis=0)
    partial_train_targets=np.concatenate([train_targets[:i*num_val_samples],train_targets[(i+1)*num_val_samples:]],axis=0)
    
    print("학습 시작....",end=' ')
    # 모델 학습
    model=build_model()
    # verbose=0 이면 훈련과정 출력되지 않는다
    history=model.fit(partial_train_data,partial_train_targets,validation_data=(val_data,val_targets),epochs=num_epochs,batch_size=1,verbose=0)
    print("학습 끝!")
    
    # 모델 평가
    mae_history=history.history['val_mean_absolute_error']
    all_mae_histories.append(mae_history)